In [2]:
import json
import pickle
import re
import requests

from IPython.display import HTML, display
import numpy as np
from pyrfume import pubchem
from rickpy import ProgressBar
import seaborn as sns

In [155]:
start = cid
finish = int(1e5)
p = ProgressBar(finish-start)
for cid in range(start, finish):
    p.animate(cid-start)
    try:
        summary = pubchem.get_summary(cid)
    except requests.exceptions.SSLError:
        print("SSLError for CID %d" % cid)
    else:
        statements = pubchem.parse_summary_for_odor(summary)
        if statements:
            #print(cid, statements)
            summ_statements[cid] = statements

[-----------------------99%----------------------] 74071 out of 74072 complete

In [160]:
start = cid
finish = int(1e5)
p = ProgressBar(finish-start)
for cid in range(start, finish):
    p.animate(cid-start)
    try:
        phys = pubchem.get_physical_description(cid)
    except requests.exceptions.SSLError:
        print("SSLError for CID %d" % cid)
    else:
        statements = pubchem.parse_physical_description_for_odor(phys)
        if statements:
            #print(cid, statements)
            phys_statements[cid] = statements

[-----------------------99%----------------------] 84061 out of 84062 complete

In [161]:
summ_cids = set(key for key in summ_statements)
phys_cids = set(key for key in phys_statements)
len(summ_cids), len(phys_cids), len(summ_cids.difference(phys_cids)), len(phys_cids.difference(summ_cids))

(1170, 1758, 174, 762)

In [162]:
all_statements = {}
for cid in set(summ_statements) | set(phys_statements):
    summ_statements_cid = summ_statements.get(cid, [])
    phys_statements_cid = phys_statements.get(cid, [])
    all_statements[cid] = summ_statements_cid + phys_statements_cid

In [171]:
def color(cid, s, code):
    return "<p style='color: %s'>%d: %s</p>" % (code, cid, s)

html = ""
for cid in sorted(all_statements):
    statements = all_statements[cid]
    odor = 0
    odorless = 0
    for statement in statements:
        statement = statement.lower()
        if re.findall('no odor', statement):
            html += color(cid, statement, '#00BB00')
        elif re.findall('no odour', statement):
            html += color(cid, statement, '#00BB00')
        elif re.findall('no smell', statement):
            html += color(cid, statement, '#00BB00')
        elif re.findall('no fragrance', statement):
            html += color(cid, statement, '#00BB00')
        elif re.findall('odorless', statement):
            html += color(cid, statement, '#00BB00')
        elif re.findall('odourless', statement):
            html += color(cid, statement, '#00BB00')
        elif re.findall('odoratus', statement):
            html += color(cid, statement, '#000000')
        elif re.findall('sense of smell', statement):
            html += color(cid, statement, '#DD0000')
        elif re.findall('odor', statement):
            html += color(cid, statement, '#DD0000')
        elif re.findall('odour', statement):
            html += color(cid, statement, '#DD0000')
        elif re.findall('smell', statement):
            html += color(cid, statement, '#DD0000')
        elif re.findall('fragrance', statement):
            html += color(cid, statement, '#DD0000')
        elif re.findall('aroma ', statement):
            html += color(cid, statement, '#DD0000')
        else:
            html += color(cid, statement, '#000000')
    #if odor and odorless:
    #    print(statements)
    
#display(HTML(html))
with open('../data/pubchem_scrape_100000.html', 'w') as f:
    f.write(html)

In [170]:
with open('../data/pubchem_scrape_100000.pkl', 'wb') as f:
    pickle.dump(all_statements, f)